# 데이터 전처리
1. 데이터프레임으로 변환
2. 피처 엔지니어링
3. 전처리된 데이터를 저장

In [1]:
import os
import json
import h5py
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

RAW_DATA_DIR = "../input/raw_data" # 카카오에서 다운로드 받은 데이터의 디렉터리
PROCESSED_DATA_DIR = '../input/processed' # 전처리된 데이터가 저장될 디렉터리
VOCAB_DIR = os.path.join(PROCESSED_DATA_DIR, 'vocab') # 전처리에 사용될 사전 파일이 저장될 디렉터리

# 학습에 사용될 파일 리스트
train_file_list = [
    "train.chunk.01",
    "train.chunk.02"
    #"train.chunk.03",
    #"train.chunk.04",
    #"train.chunk.05",
    #"train.chunk.06",
    #"train.chunk.07",
    #"train.chunk.08",
    #"train.chunk.09"
]

# 개발에 사용될 파일 리스트. 공개 리더보드 점수를 내는데 사용된다.
dev_file_list = [
    "dev.chunk.01"    
]

# 테스트에 사용될 파일 리스트. 파이널 리더보드 점수를 내는데 사용된다.
test_file_list = [
    "test.chunk.01"
    #"test.chunk.02", 
]

# 파일명과 실제 파일이 위치한 디렉토리를 결합한다.
train_path_list = [os.path.join(RAW_DATA_DIR, fn) for fn in train_file_list]
dev_path_list = [os.path.join(RAW_DATA_DIR, fn) for fn in dev_file_list]
test_path_list = [os.path.join(RAW_DATA_DIR, fn) for fn in test_file_list]

# PROCESSED_DATA_DIR과 VOCAB_DIR를 생성한다.
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)
os.makedirs(VOCAB_DIR, exist_ok=True)

## 1. 데이터프레임으로 변환
HDF5 포맷의 대회 파일로부터 데이터를 읽어서 Pandas의 DataFrame으로 변환하는 함수 생성

In [2]:
# path_list의 파일에서 col 변수에 해당하는 컬럼 값들을 가져온다.
def get_column_data(path_list, div, col):
    col_data = []
    for path in path_list:
        h = h5py.File(path, 'r')
        col_data.append(h[div][col][:])
        h.close()
    return np.concatenate(col_data)

# path_list의 파일에서 학습에 필요한 컬럼들을 DataFrame 포맷으로 반환한다.
def get_dataframe(path_list, div):
    pids = get_column_data(path_list, div, col='pid')
    products = get_column_data(path_list, div, col='product')
    brands = get_column_data(path_list, div, col='brand')
    makers = get_column_data(path_list, div, col='maker')
    # 16GB를 가진 PC에서 실행 가능하도록 메모리를 많이 사용하는 칼럼은 주석처리
    #models = get_column_data(path_list, div, col='model') 
    prices = get_column_data(path_list, div, col='price')
    updttms = get_column_data(path_list, div, col='updttm')
    bcates = get_column_data(path_list, div, col='bcateid')
    mcates = get_column_data(path_list, div, col='mcateid')
    scates = get_column_data(path_list, div, col='scateid')
    dcates = get_column_data(path_list, div, col='dcateid')
    
    df = pd.DataFrame({'pid': pids, 'product':products, 'brand':brands, 'maker':makers, 
                                      #'model':models, 
                                      'price':prices, 'updttm':updttms, 
                                      'bcateid':bcates, 'mcateid':mcates, 'scateid':scates, 'dcateid':dcates} )
    
    # 바이트 열로 인코딩된 상품제목과 상품ID를 유니코드 변환한다.
    df['pid'] = df['pid'].map(lambda x: x.decode('utf-8'))
    df['product'] = df['product'].map(lambda x: x.decode('utf-8'))
    df['brand'] = df['brand'].map(lambda x: x.decode('utf-8'))
    df['maker'] = df['maker'].map(lambda x: x.decode('utf-8'))
    #df['model'] = df['model'].map(lambda x: x.decode('utf-8')) # 메모리 사용량을 줄이기 위해 주석처리
    df['updttm'] = df['updttm'].map(lambda x: x.decode('utf-8'))     
    
    return df

get_dataframe() 함수를 사용해서 raw data를 데이터프레임 자료형 변수로 변환

In [3]:
train_df = get_dataframe(train_path_list, 'train')
dev_df = get_dataframe(dev_path_list, 'dev')
test_df = get_dataframe(test_path_list, 'test')

shape을 출력해서 정상적으로 생성됐는지 확인

In [4]:
train_df.shape

(2000000, 10)

In [5]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 500)
train_df.head(3)

,pid,product,brand,maker,price,updttm,bcateid,mcateid,scateid,dcateid
0,O4486751463,직소퍼즐 - 1000조각 바다거북의 여행 (PL1275),퍼즐라이프,상품상세설명 참조,16520,20180227091029,1,1,2,-1
1,P3307178849,[모리케이스]아이폰6S/6S+ tree farm101 - 다이어리케이스[바보사랑][무료배송],바보사랑,MORY|해당없음,20370,20180429085019,3,3,4,-1
2,R4424255515,크리비아 기모 3부 속바지 GLG4314P,크리비아,,-1,20180426102314,5,5,6,-1


### 칼럼 설명
- pid : 상품 고유 ID  
~
- model: 상품명을 정제 알고리즘으로 정제한 결과 (여기서는 16GB RAM의 한계로 skip)
- pupdttm : 상품정보가 업데이트된 시간
- bcateid : 대 카테고리 ID
- mcateid : 중 카테고리 ID
- scateid : 소 카테고리 ID
- dcateid : 상세 카테고리 ID
- img_feat : ResNet50 모델의 출력 결과. 이 칼럼은 용량이 커서 데이터프레임의 칼럼으로 추가하지 않고 별도의 HDF5 포맷의 파일로 저장할 예정

## 2. 피처 엔지니어링
모델의 성능 향상을 목적으로 데이터프레임을 가공

### 정답을 예측하는데 도움이 될 칼럼만 선택하기
중복된 피처나 노이즈가 섞인 피처는 제외한다. 여기서는 pid, product만 이용해보자.

### 카테고리 이름 칼럼 추가
cate1.json 파일에 매핑 정보를 사용해서 카테고리 칼럼의 값을 한글 이름으로 바꾸자.

In [6]:
# 카테고리 이름과 ID의 매핑 정보를 불러온다.
cate_json = json.load(open(os.path.join(RAW_DATA_DIR, 'cate1.json'), encoding='UTF8'))

# (이름, ID) 순서를 (ID, 이름)으로 바꾼 후 dictionary로 만든다.
bid2nm = dict([(cid, name) for name, cid in cate_json['b'].items()])
mid2nm = dict([(cid, name) for name, cid in cate_json['m'].items()])
sid2nm = dict([(cid, name) for name, cid in cate_json['s'].items()])
did2nm = dict([(cid, name) for name, cid in cate_json['d'].items()])

# dictionary를 활용해 카테고리 ID에 해당하는 카테고리 이름 컬럼을 추가한다.
train_df['bcatenm'] = train_df['bcateid'].map(bid2nm)
train_df['mcatenm'] = train_df['mcateid'].map(mid2nm)
train_df['scatenm'] = train_df['scateid'].map(sid2nm)
train_df['dcatenm'] = train_df['dcateid'].map(did2nm)

In [7]:
#train_df[['pid', 'product', 'brand', 'maker', 'model', 'price', 'bcatenm', 'mcatenm', 'scatenm', 'dcatenm']].head(5)
train_df[['pid', 'product', 'brand', 'maker','price', 'bcatenm', 'mcatenm', 'scatenm', 'dcatenm']].head(5)

,pid,product,brand,maker,price,bcatenm,mcatenm,scatenm,dcatenm
0,O4486751463,직소퍼즐 - 1000조각 바다거북의 여행 (PL1275),퍼즐라이프,상품상세설명 참조,16520,악기/취미/만들기,보드게임/퍼즐,직소/퍼즐,
1,P3307178849,[모리케이스]아이폰6S/6S+ tree farm101 - 다이어리케이스[바보사랑][무료배송],바보사랑,MORY|해당없음,20370,휴대폰/액세서리,휴대폰액세서리,아이폰액세서리,
2,R4424255515,크리비아 기모 3부 속바지 GLG4314P,크리비아,,-1,언더웨어,보정언더웨어,속바지/속치마,
3,F3334315393,[하프클럽/잭앤질]남성 솔리드 절개라인 포인트 포켓 팬츠 31133PT002_NA,잭앤질,㈜크리스패션,16280,남성의류,바지,일자면바지,
4,N731678492,코드프리혈당시험지50매/코드프리시험지/최장유효기간,,기타,-1,건강관리/실버용품,건강측정용품,혈당지,


### 칼럼별 중요성 따져 보기
1. pid : 상품의 특징과 무관하므로 타깃을 예측하는 데 도움이 안된다.
2. product : 상품명에는 직접적으로 타깃의 단어가 포함돼 있다.
3. brand : 카테고리와 관련은 있지만 공백값이 존재한다. 조사해보자.

In [8]:
def get_vc_df(df, col):    
    vc_df = df[col].value_counts().reset_index()
    vc_df.columns = [col, 'count']
    vc_df['percentage'] = (vc_df['count'] / vc_df['count'].sum())*100    
    return vc_df

vc_df = get_vc_df(train_df, 'brand')
vc_df.head(10)

,brand,count,percentage
0,,582255,29.11275
1,바보사랑,26014,1.30070
2,상품상세설명 참조,21480,1.07400
3,아디다스,15082,0.75410
4,아트박스,12601,0.63005
5,오가닉맘,12246,0.61230
6,나이키,9736,0.48680
7,기타,9242,0.46210
8,꾸밈,8185,0.40925
9,없음,7867,0.39335


공백문자가 무려 30%나 차지한다. '상품상세설명 참조', '없음' 등 타깃 예측에 도움 안 되는 값도 많다.

4. maker : 마찬가지로 도움 안 되는 데이터가 많다.

In [9]:
vc_df = get_vc_df(train_df, 'maker')
vc_df.head(10)

,maker,count,percentage
0,기타,416744,20.83720
1,,325497,16.27485
2,상품상세설명 참조,60345,3.01725
3,아디다스,13596,0.67980
4,LF,11360,0.56800
5,꾸밈,10736,0.53680
6,[불명],7762,0.38810
7,나이키,7423,0.37115
8,La Diosa,7200,0.36000
9,삼성물산 패션부문,6725,0.33625


5. model : 상품명을 정제 알고리즘으로 정제한 결과이다. 카카오의 자체 알고리즘으로 추출된 데이터로 보인다.

In [10]:
# 메모리 사용량을 줄이기 위해 주석처리
#vc_df = get_vc_df(train_df, 'model')
#vc_df.head(10)

6. price : -1 값(가격 정보 없음)이 상당수 포함돼 있다.

In [11]:
vc_df = get_vc_df(train_df, 'price')
vc_df.head(10)

,price,count,percentage
0,-1,1263578,63.17890
1,9900,1070,0.05350
2,14880,1041,0.05205
3,12000,1029,0.05145
4,19800,974,0.04870
5,18000,924,0.04620
6,9000,897,0.04485
7,14000,890,0.04450
8,14400,886,0.04430
9,19000,858,0.04290


### 불필요한 칼럼을 제외한 새로운 데이터프레임 만들기
- brand, maker, model 칼럼은 절반 정도의 데이터에는 타깃 예측에 필요한 정보가 담겨있지 않다.
- brand, maker, model 칼럼은 product에 이미 포함된 정보인 경우가 많으므로 제외하자.
- price 칼럼은 가격 정보가 없는 비중이 크지만, product에 없는 정보이므로 도움이 될 수도 있다. 이런 경우 포함했을 때와 미포함했을 때의 예측 결과를 비교해 판단해볼수 있다.

In [12]:
train_df = train_df[['pid', 'product', 'bcateid', 'mcateid', 'scateid', 'dcateid']]
dev_df = dev_df[['pid', 'product', 'bcateid', 'mcateid', 'scateid', 'dcateid']]
test_df = test_df[['pid', 'product', 'bcateid', 'mcateid', 'scateid', 'dcateid']]

In [13]:
train_df.head()

,pid,product,bcateid,mcateid,scateid,dcateid
0,O4486751463,직소퍼즐 - 1000조각 바다거북의 여행 (PL1275),1,1,2,-1
1,P3307178849,[모리케이스]아이폰6S/6S+ tree farm101 - 다이어리케이스[바보사랑][무료배송],3,3,4,-1
2,R4424255515,크리비아 기모 3부 속바지 GLG4314P,5,5,6,-1
3,F3334315393,[하프클럽/잭앤질]남성 솔리드 절개라인 포인트 포켓 팬츠 31133PT002_NA,7,7,8,-1
4,N731678492,코드프리혈당시험지50매/코드프리시험지/최장유효기간,10,9,11,-1


### product  칼럼 전처리

**칼럼의 특징**
- 상품명은 한 문장으로 취급한다.
- 브랜드, 제품명, 제품코드 등 명사의 나열이다.
- 띄어쓰기가 없거나 특수기호로 단어를 붙이는 경우가 많다. ex) 코드프리혈당시험지50매/코드프리시험지/최장유효기간
- 특수기호를 특정 단어를 강조하기 위해 반복해서 쓴다. ex) [하프클럽/잭앤질]

1. 딥러닝 모델 입력에 불필요한 특수기호 제거

In [14]:
import re

# 특수기호를 나열한 패턴 문자열을 컴파일하여 패턴 객체를 얻는다.
p = re.compile('[\!@#$%\^&\*\(\)\-\=\[\]\{\}\.,/\?~\+\'"|_:;><`┃]')

# 위의 패턴 문자열의 매칭되는 문자는 아래 코드를 통해서 빈공백으로 치환할 것이다.

# 문장의 특수기호 제거 함수
def remove_special_characters(sentence, lower=True):
    sentence = p.sub(' ', sentence) # 패턴 객체로 sentence 내의 특수기호를 공백문자로 치환한다.
    sentence = ' '.join(sentence.split()) # sentence 내의 두개 이상 연속된 빈공백들을 하나의 빈공백으로 만든다.
    if lower:
        sentence = sentence.lower()
    return sentence

# product 칼럼에 특수기호를 제거하는 함수를 적용한 결과를 반환한다.
train_df['product'] = train_df['product'].map(remove_special_characters)

train_df.head() # 특수기호가 제거된 train_df의 상단 5행만 출력

,pid,product,bcateid,mcateid,scateid,dcateid
0,O4486751463,직소퍼즐 1000조각 바다거북의 여행 pl1275,1,1,2,-1
1,P3307178849,모리케이스 아이폰6s 6s tree farm101 다이어리케이스 바보사랑 무료배송,3,3,4,-1
2,R4424255515,크리비아 기모 3부 속바지 glg4314p,5,5,6,-1
3,F3334315393,하프클럽 잭앤질 남성 솔리드 절개라인 포인트 포켓 팬츠 31133pt002 na,7,7,8,-1
4,N731678492,코드프리혈당시험지50매 코드프리시험지 최장유효기간,10,9,11,-1


2. product 칼럼의 문장의 단어를 분절하는 tokenizer

일반 문장과 다른 상품명의 특성 때문에 konlp의 한국어 형태소 분석기는 쓰지 않는다.  
최근 딥러닝 기반 NLP 분야에서 자주 사용되는 학습 기반의 문장 tokenizer인 구글의 sentencepice 라이브러리를 사용해보자.
- sentencepice : subword tokenizer (https://wikidocs.net/86657)

In [15]:
%%time
import sentencepiece as spm # sentencepiece 모듈을 가져온다.

# product 칼럼의 상품명을 product.txt 파일명으로 저장한다.
with open(os.path.join(VOCAB_DIR, 'product.txt'), 'w', encoding='utf-8') as f:
    f.write(train_df['product'].str.cat(sep='\n'))

# sentencepiece 모델을 학습시키는 함수이다.
def train_spm(txt_path, spm_path,
              vocab_size=32000, input_sentence_size=1000000):  
    # input_sentence_size: 개수 만큼만 학습데이터로 사용된다.
    # vocab_size: 사전 크기
    spm.SentencePieceTrainer.Train(
        f' --input={txt_path} --model_type=bpe'
        f' --model_prefix={spm_path} --vocab_size={vocab_size}'
        f' --input_sentence_size={input_sentence_size}'
        f' --shuffle_input_sentence=true'
    )

# product.txt 파일로 sentencepiece 모델을 학습 시킨다. 
# 학습이 완료되면 spm.model, spm.vocab 파일이 생성된다.
train_spm(txt_path=os.path.join(VOCAB_DIR, 'product.txt'), 
          spm_path=os.path.join(VOCAB_DIR, 'spm')) # spm 접두어

# 센텐스피스 모델 학습이 완료되면 product.txt는 삭제
os.remove(os.path.join(VOCAB_DIR, 'product.txt'))

# 필요한 파일이 제대로 생성됐는지 확인
for dirname, _, filenames in os.walk(VOCAB_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/processed\vocab\spm.model
../input/processed\vocab\spm.vocab
Wall time: 1min 29s


학습된 모델은 파일로 저장된다. (spm.model)  
이 모델을 활용해 product 칼럼을 분절해보자.

In [16]:
# 센텐스피스 모델을 로드한다.
sp = spm.SentencePieceProcessor()
sp.Load(os.path.join(VOCAB_DIR, 'spm.model'))

# product 칼럼의 상품명을 분절한 결과를 tokenized_product 칼럼에 저장한다.
train_df['tokens'] = train_df['product'].map(lambda x: " ".join(sp.EncodeAsPieces(x)) )

train_df[['product', 'tokens']].head()

,product,tokens
0,직소퍼즐 1000조각 바다거북의 여행 pl1275,▁직소퍼즐 ▁1000 조각 ▁바다 거북 의 ▁여행 ▁pl 1275
1,모리케이스 아이폰6s 6s tree farm101 다이어리케이스 바보사랑 무료배송,▁모리케이스 ▁아이폰 6 s ▁6 s ▁tree ▁farm 101 ▁다이어리케이스 ▁바보사랑 ▁무료배송
2,크리비아 기모 3부 속바지 glg4314p,▁크리비아 ▁기모 ▁3 부 ▁속바지 ▁gl g 43 14 p
3,하프클럽 잭앤질 남성 솔리드 절개라인 포인트 포켓 팬츠 31133pt002 na,▁하프클럽 ▁잭앤질 ▁남성 ▁솔리드 ▁절개라인 ▁포인트 ▁포켓 ▁팬츠 ▁3 1133 pt 002 ▁na
4,코드프리혈당시험지50매 코드프리시험지 최장유효기간,▁코드 프리 혈 당 시험 지 50 매 ▁코드 프리 시험 지 ▁최 장 유 효 기간


train_df의 product 칼럼으로 학습된 분절기를 그대로 사용해서 dev_df와 test_df도 전처리

In [17]:
# 특수기호를 공백문자로 치환
dev_df['product'] = dev_df['product'].map(remove_special_characters) 
# product 칼럼을 분절한 뒤 token_id로 치환
dev_df['tokens'] = dev_df['product'].map(lambda x: " ".join([str(token_id) for token_id in sp.EncodeAsPieces(x)]))

# 특수기호를 공백문자로 치환
test_df['product'] = test_df['product'].map(remove_special_characters) 
# product 칼럼을 분절한 뒤 token_id로 치환
test_df['tokens'] = test_df['product'].map(lambda x: " ".join([str(token_id) for token_id in sp.EncodeAsPieces(x)]))

In [18]:
dev_df[['product', 'tokens']].head()

,product,tokens
0,gigabyte 미니pc gb bace 3160 램 4g hdd 500gb w,▁gigabyte ▁미니 pc ▁gb ▁b ace ▁3 160 ▁램 ▁4 g ▁hdd ▁500 gb ▁w
1,와코루 wacoal 와코루 튤레이스 홑겹 b컵브라 2칼라 nb sp dbr0156,▁와코루 ▁wacoal ▁와코루 ▁튤레이스 ▁홑겹 ▁b 컵브라 ▁2 칼라 ▁nb ▁sp ▁dbr 01 56
2,카렉스 블랙스2 핸들커버 실버 아반떼xd,▁카렉스 ▁블랙스 2 ▁핸들커버 ▁실버 ▁아반떼 xd
3,뉴에라 mlb 도트 프린트 뉴욕 양키스 티셔츠 화이트 11502825,▁뉴에라 ▁mlb ▁도트 ▁프린트 ▁뉴욕 ▁양키스 ▁티셔츠 ▁화이트 ▁115 028 25
4,플러그피트니스 네오플랜 삼각아령5kg 아령 여자아령 여성아령 팔운동 여성덤벨 frog,▁플러그 피 트니스 ▁네오플랜 ▁삼각 아령 5 kg ▁아령 ▁여자 아령 ▁여성 아령 ▁팔 운동 ▁여성 덤벨 ▁frog


## 3. 전처리된 데이터를 저장

CSV 포맷의 파일로 저장

In [20]:
# product, tokenized_product 칼럼을 제외한 칼럼만을 남긴다.
columns = ['pid', 'tokens',  'bcateid', 'mcateid', 'scateid', 'dcateid']
train_df = train_df[columns] 
dev_df = dev_df[columns] 
test_df = test_df[columns] 

# csv 포맷으로 저장한다.
train_df.to_csv(os.path.join(PROCESSED_DATA_DIR, 'train.csv'), index=False) 
dev_df.to_csv(os.path.join(PROCESSED_DATA_DIR, 'dev.csv'), index=False) 
test_df.to_csv(os.path.join(PROCESSED_DATA_DIR, 'test.csv'), index=False) 

## img_feat 데이터 전처리 및 저장
img_feat 칼럼은 크기가 크고 데이터프레임으로 처리할 수 없는 벡터이므로 별도로 처리한다.  
h5 포맷의 데이터에서 img_feat만 분리해서 h5 포맷의 파일로 저장한다.  
이 피쳐는 상품의 이미지이므로 상품명(tokens)와 함께 중요한 칼럼이다.

In [21]:
# image_feature는 데이터의 크기가 크므로 처리함수를 별도로 분리하였다.
def save_column_data(input_path_list, div, col, n_img_rows, output_path):
    # img_feat를 저장할 h5 파일을 생성
    h_out = h5py.File(output_path, 'w')    
    # 대회데이터의 상품개수 x 2048(img_feat 크기)로 dataset을 할당한다.
    h_out.create_dataset(col, (n_img_rows, 2048), dtype=np.float32)
    
    offset_out = 0
    
    # h5포맷의 대회데이터에서 img_feat 칼럼만 읽어서 h5포맷으로 다시 저장한다.
    for in_path in tqdm(input_path_list, desc=f'{div},{col}'):
        h_in = h5py.File(in_path, 'r')
        sz = h_in[div][col].shape[0]
        h_out[col][offset_out:offset_out+sz] = h_in[div][col][:]
        offset_out += sz
        h_in.close()
    h_out.close()


save_column_data(train_path_list, div='train', col='img_feat', n_img_rows=len(train_df), 
                 output_path=os.path.join(PROCESSED_DATA_DIR, 'train_img_feat.h5'))
save_column_data(dev_path_list, div='dev', col='img_feat', n_img_rows=len(dev_df), 
                 output_path=os.path.join(PROCESSED_DATA_DIR, 'dev_img_feat.h5'))
save_column_data(test_path_list, div='test', col='img_feat', n_img_rows=len(test_df), 
                 output_path=os.path.join(PROCESSED_DATA_DIR, 'test_img_feat.h5'))

# 파일이 제대로 생성됐는지 확인
for dirname, _, filenames in os.walk(PROCESSED_DATA_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))

train,img_feat:   0%|          | 0/2 [00:00<?, ?it/s]

dev,img_feat:   0%|          | 0/1 [00:00<?, ?it/s]

test,img_feat:   0%|          | 0/1 [00:00<?, ?it/s]

../input/processed\dev.csv
../input/processed\dev_img_feat.h5
../input/processed\test.csv
../input/processed\test_img_feat.h5
../input/processed\train.csv
../input/processed\train_img_feat.h5
../input/processed\vocab\spm.model
../input/processed\vocab\spm.vocab
